<a href="https://colab.research.google.com/github/Mohamed-Hedi-Jemaa/Tunisian-Fraud-Detection-Challenge/blob/main/Tunisian_Fraud_Detection_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb

In [ ]:
import math
rmse = lambda x, y: math.sqrt(mean_squared_error(x, y))

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Zindi/SUPCOM_Train.csv")
train.set_index('id', inplace=True)
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Zindi/SUPCOM_Test.csv")
test.set_index('id', inplace=True)

In [ ]:
train.head()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,...,RES_ANNIMP,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C
id,,,,,,,,,,,,,,,,,,,,,
train_id17437,44,6210,C,99,0,3707,4,2,2,2,...,2014,16.639203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id5086,401,14383,M,22,0,6105,4,2,2,2,...,2013,17.827703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id1443,243,11555,M,23,0,5751,4,2,2,2,...,2013,15.473503,25387000.0,0.0,0.0,4571280.0,NaN,NaN,NaN,NaN
train_id15469,72,6175,C,99,0,4402,4,2,2,2,...,2017,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id14368,42,1417,M,23,0,6302,4,2,2,2,...,2014,14.286244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train['target'].head()

id
train_id17437    16.639203
train_id5086     17.827703
train_id1443     15.473503
train_id15469     0.000000
train_id14368    14.286244
Name: target, dtype: float64

In [ ]:
train['is_train'] = 0
test['is_train'] = 1
train_y = train['target']

In [ ]:
train = train.drop('target',axis=1)

In [ ]:
df = pd.concat([train, test], axis=0)

In [ ]:
df

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,...,RES_ANNIMP,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,is_train
id,,,,,,,,,,,,,,,,,,,,,
train_id17437,44,6210,C,99,0,3707,4,2,2,2,...,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id5086,401,14383,M,22,0,6105,4,2,2,2,...,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id1443,243,11555,M,23,0,5751,4,2,2,2,...,2013,25387000.0,0.0,0.0,4571280.0,NaN,NaN,NaN,NaN,0
train_id15469,72,6175,C,99,0,4402,4,2,2,2,...,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id14368,42,1417,M,23,0,6302,4,2,2,2,...,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test_id1497,193,1162,P,99,0,6307,4,2,2,2,...,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
test_id10025,68,5572,M,22,0,3802,4,2,2,2,...,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
test_id1045,173,2943,M,22,0,4221,4,2,2,2,...,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
df.head()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,...,RES_ANNIMP,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,is_train
id,,,,,,,,,,,,,,,,,,,,,
train_id17437,44,6210,C,99,0,3707,4,2,2,2,...,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id5086,401,14383,M,22,0,6105,4,2,2,2,...,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id1443,243,11555,M,23,0,5751,4,2,2,2,...,2013,25387000.0,0.0,0.0,4571280.0,NaN,NaN,NaN,NaN,0
train_id15469,72,6175,C,99,0,4402,4,2,2,2,...,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id14368,42,1417,M,23,0,6302,4,2,2,2,...,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
df['CTR_CATEGO_X'].value_counts()

M    14862
C     7735
P     6207
N        8
Name: CTR_CATEGO_X, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
df['CTR_CATEGO_X'] = LabelEncoder().fit_transform(df['CTR_CATEGO_X'])

In [ ]:
df = df.apply(lambda x: x.fillna(x.median()),axis=0)

In [ ]:
train_x = df

In [ ]:
test_x = train_x[train_x['is_train'] == 1]
train_x = train_x[train_x['is_train'] == 0]
train_x = train_x.drop(['is_train',],axis=1)
test_x = test_x.drop(['is_train',],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt


ModuleNotFoundError: ignored

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from multiprocessing.pool import Pool
from catboost import Pool, CatBoostRegressor

In [ ]:
#pool = Pool()
train_pool = Pool(train_x, train_y)
test_pool = Pool(test_x)
cbm0 = CatBoostRegressor(rsm=0.2, depth=13, learning_rate=0.2, eval_metric='RMSE')

In [ ]:
#XGBRegressor
boost_params = {'eval_metric': 'rmse'}
xgb0 = xgb.XGBRegressor(
    max_depth=13,
    learning_rate=0.1,
    n_estimators=2000,
    objective='reg:linear',
    gamma=0,
    min_child_weight=1,
    subsample=1,
    colsample_bytree=1,
    scale_pos_weight=1,
    seed=4,
    **boost_params)

#LGBMRegressor
gbm0 = lgb.LGBMRegressor(
    objective='regression',
    num_leaves=65,
    learning_rate=0.2,
    n_estimators=2000)

print("Fitting CatBoostRegressor model...")
cbm0.fit(train_pool)
print("Finished fitting CatBoostRegressor model")

print("Fitting XGBRegressor model...")
xgb0.fit(train_x, train_y)
print("Finished fitting XGBRegressor model")

print("Fitting LGBMRegressor model...")
gbm0.fit(train_x, train_y, eval_metric='rmse')
print("Finished fitting LGBMRegressor model")

Fitting CatBoostRegressor model...
0:	learn: 6.8255341	total: 374ms	remaining: 6m 14s
1:	learn: 6.6541350	total: 916ms	remaining: 7m 36s
2:	learn: 6.4974100	total: 1.39s	remaining: 7m 40s
3:	learn: 6.3733176	total: 1.84s	remaining: 7m 37s
4:	learn: 6.2640264	total: 2.2s	remaining: 7m 17s
5:	learn: 6.2021097	total: 2.6s	remaining: 7m 11s
6:	learn: 6.1263210	total: 3.02s	remaining: 7m 8s
7:	learn: 6.0664084	total: 3.33s	remaining: 6m 53s
8:	learn: 6.0163585	total: 3.69s	remaining: 6m 46s
9:	learn: 5.9524306	total: 4.16s	remaining: 6m 52s
10:	learn: 5.9138402	total: 4.67s	remaining: 6m 59s
11:	learn: 5.8692884	total: 5.03s	remaining: 6m 54s
12:	learn: 5.8266463	total: 5.39s	remaining: 6m 48s
13:	learn: 5.7914716	total: 5.85s	remaining: 6m 52s
14:	learn: 5.7349717	total: 6.3s	remaining: 6m 53s
15:	learn: 5.7049248	total: 6.66s	remaining: 6m 49s
16:	learn: 5.6647196	total: 7.15s	remaining: 6m 53s
17:	learn: 5.6288105	total: 7.56s	remaining: 6m 52s
18:	learn: 5.6124605	total: 8.15s	remaining

In [ ]:
predict_y1 = gbm0.predict(test_x)
predict_y2 = xgb0.predict(test_x)
predict_y3 = cbm0.predict(test_pool)

In [ ]:
predict_y1

array([16.18727035, 17.1593332 ,  5.24365835, ..., 18.58356353,
       15.58920561, 12.55627711])

In [ ]:
predict_y = (predict_y1 + predict_y2+predict_y3)/3

In [ ]:
predict_y

array([16.49153301, 17.11337563,  8.2512268 , ..., 17.32305709,
       13.70606927, 13.67952698])

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
submit = pd.DataFrame(
   {
       'id':test.index,
       'target':predict_y
   }
)

submit.to_csv('submit.csv',index=False)
!cp submit.csv drive/My\ Drive/

In [ ]:
final = pd.read_csv("submit.csv")
final.head()

,id,target
0,test_id1120,16.491533
1,test_id1680,17.113376
2,test_id1063,8.251227
3,test_id3731,14.978874
4,test_id9766,14.091756
